In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import numpy 
import math
import cv2
from os import walk
import os 
import torch 
import torch 
import torchvision 

In [87]:
class AlexNet_metric():
  def __init__(self):
      self.net = torchvision.models.alexnet(pretrained=True)
      self.net.classifier = torch.nn.Sequential()
      print(self.net)

  def calculate(self, image):
      return self.net(image)

a = AlexNet_metric()

print(a.net)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential()
)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), paddin

In [94]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


metric = AlexNet_metric()
cosinesim = torch.nn.CosineSimilarity(dim=1)


# CREDITS: Himanshu Tyagi @ Stack OVerflow 

def psnr(img1, img2):

  if (img1.shape != img2.shape):
    w,h, c = img1.shape
    img2 = cv2.resize(img2, (h,w ))

  mse = numpy.mean( (img1 - img2) ** 2 )
  if mse == 0:
    return 100
  PIXEL_MAX = 255.0
  return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))


def feature_diff(img1, img2 ):

  transform = torchvision.transforms.ToTensor()
  feat1 = metric.calculate(transform(img1).to(device).unsqueeze(0))
  feat2 = metric.calculate(transform(img2).to(device).unsqueeze(0))
  return cosinesim(feat1, feat2).item()



%cd /content/drive/MyDrive/CSC413/set5
_, _, filenames = next(walk("."))

filenames = [filename for filename in filenames if 'GAN' not in filename 
             and 'pred' not in filename 
             and 'Bicubic' not in filename
             and 'cropped' not in filename]



for name in filenames:
  filename = os.path.splitext(name)[0]
  GAN_path = filename + "_GAN.png"
  MEM_path = "pred_" + filename + ".jpg"
  original = filename + ".png"

  GAN = cv2.imread(GAN_path)
  MEM = cv2.imread(MEM_path)
  original = cv2.imread(original)

  PSNR_GAN = feature_diff(original, GAN)
  PSNR_MEM = feature_diff(original, MEM)

  print( filename + " : GAN:", end ="")
  print(PSNR_GAN, end="")
  print(": MEM:", end="")
  print(PSNR_MEM)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential()
)
/content/drive/MyDrive/CSC413/set5
bird : GAN:0.9489492774009705: MEM:0.9603338241577148
butterfly : 